# Cluster onboarding
This guide explains how you can onboard to one of the Operate First clusters. First chapter lists different environments and clusters where you can onboard, second explains how you can onboard as user and third how you can onboard yourself creating PR.
:::{seealso}
For more information about the Operate First see [Getting started with Operate First](references:operate-first-start)
:::

## User

Clusters managed by Operate First are listed [here](references:clusters).
To request access to one of these clusters please file a GitHub issue [here](https://github.com/operate-first/support/issues/new?assignees=first-operator&labels=kind%2Fonboarding%2Carea%2Fcluster&template=onboarding_to_cluster.yaml&title=NEW+PROJECT%3A+%3Cname%3E). 


## Operation

This part of tutorial servers as a guide for adding cluster scoped resources of `Namespace` and `Group` kinds. Following steps in this guide should result in a PR against the [operate-first/apps](https://github.com/operate-first/apps) repository.

### Prerequisites
There are no prerequisites.

### Recipe
If you want to know more about the overall design please consult the ADR documentation at [operate-first/support](https://github.com/operate-first/blueprint).
In general we store all the cluster-scoped resources in a `cluster-scope` kustomize application within this repository.
#### Workspace setup

Please fork/clone the [operate-first/apps](https://github.com/operate-first/apps) repository. During this whole setup, we’ll be working within this repository.

1. Go to [operate-first/apps](https://github.com/operate-first/apps).
2. Click on a fork button.
3. When a fork is created click on the code button and copy an adress of your forked repository.
4. Run following command using copied adress:

In [43]:
!git clone https://github.com/$JUPYTERHUB_USER/apps.git
%cd apps

Cloning into 'apps'...
remote: Enumerating objects: 9615, done.
remote: Counting objects: 100% (3915/3915), done.
remote: Compressing objects: 100% (1847/1847), done.
remote: Total 9615 (delta 2166), reused 3341 (delta 1884), pack-reused 5700
Receiving objects: 100% (9615/9615), 2.19 MiB | 2.37 MiB/s, done.
Resolving deltas: 100% (4650/4650), done.
/home/mdrla/work/operate-first/hitchhikers-guide/apps


#### Define important variables
Define variables which are used later in the tutorial.

In [44]:
NAMESPACE_NAME="my-namespace"
NAMESPACE_DISPLAY_NAME="My-namespace"
TEAM_NAME="awesome-team"
TARGET_CLUSTER="moc/zero"


#### Adding namespaces
For easier onboard to ArgoCD later on, we prefer to follow a name pattern for all our namespaces. Please use your team name as a prefix to the namespace name like so: `$TEAM_NAME-example-project`.
#####  Base resources


To create necessary resources use opfcli which creates the following folders with files:

- A namespace in `cluster-scope/base/core/namespaces/$NAMESPACE_NAME`
- A blank user group for the `$TEAM_NAME` if it does not exist yet in the `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME`
- An RBAC component for the project admin role `RoleBinding` in `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME` and maps it to the newly added namespace.

In [45]:
!opfcli create-project $NAMESPACE_NAME $TEAM_NAME -d $NAMESPACE_DISPLAY_NAME

INFO[0000] writing group definition to /home/mdrla/work/operate-first/hitchhikers-guide/apps/cluster-scope/base/user.openshift.io/groups/awesome-team 
INFO[0000] writing rbac definition to /home/mdrla/work/operate-first/hitchhikers-guide/apps/cluster-scope/components/project-admin-rolebindings/awesome-team 
INFO[0000] writing namespace definition to /home/mdrla/work/operate-first/hitchhikers-guide/apps/cluster-scope/base/core/namespaces/my-namespace 



##### Enabling namespace deployment to Zero cluster


In [26]:
!cd cluster-scope/overlays/prod/$TARGET_CLUSTER

Open the `kustomization.yaml` file in this folder and add `../../base/core/namespaces/$NAMESPACE` to the `resources` field. We aim to keep this field alphabetically sorted for better human readability:
```
---
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization

resources:
...
- ../../base/core/namespaces/$NAMESPACE
...
```
#### Populate your team
The following steps enable users to access designated cluster/namespaces. This consists of adding users to the appropriate OpenShift groups.

Navigate to `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/`.


In [27]:
!cd cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/

Edit group.yaml file by adding GitHub username of every user in the group.
```
---
apiVersion: user.openshift.io/v1
kind: Group
metadata:
  name: $TEAM_NAME
users:
  - user_1
  - user_2
```
```{note}
All usernames **must** be converted to lowercase!
```

#### Finalize
Please stage your changes and send them as a PR against the [operate-first/apps](https://github.com/operate-first/apps) repository. 
:::{note}
Make sure that following files/ have been modified/added:
- [x] `cluster-scope/base/core/namespaces/$NAMESPACE_NAME/kustomization.yaml`
- [x] `cluster-scope/base/core/namespaces/$NAMESPACE_NAME/namespace.yaml`
- [x] `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/group.yaml`
- [x] `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/kustomization.yaml`
- [x] `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME/kustomization.yaml`
- [x] `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME/rbac.yaml`
- [x] `cluster-scope/overlays/prod/$TARGET_CLUSTER/kustomization.yaml`
:::

In [46]:
!git status
!git add .
!git commit -m "feat(onboarding): Add team $TEAM_NAME"
!git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   cluster-scope/overlays/prod/moc/zero/kustomization.yaml

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cluster-scope/base/core/namespaces/my-namespace/
	cluster-scope/base/user.openshift.io/groups/awesome-team/
	cluster-scope/components/project-admin-rolebindings/awesome-team/

no changes added to commit (use "git add" and/or "git commit -a")
[master 1d0c875] feat(onboarding): Add team awesome-team
 7 files changed, 41 insertions(+)
 create mode 100644 cluster-scope/base/core/namespaces/my-namespace/kustomization.yaml
 create mode 100644 cluster-scope/base/core/namespaces/my-namespace/namespace.yaml
 create mode 100644 cluster-scope/base/user.openshift.io/groups/awesome-team/group.yaml
 create mode 100644 clust

##### Authenticate via OpenShift
OpenShift `User` resources are automatically created upon user login into the cluster. Please see the list of available clusters [here](references:clusters) to get the appropriate OpenShift console link.
This will lead the user to an authentication provider selection screen:

```{image} ../img/openshift-login.png
:alt: openshift-login
:align: center
```
<br/><br/>
Please select `operate-first` provider.